In [1]:
## The neuropath data at Emory ADRC is currently not particularly well organized
## and naming schema is inconsistent.  Working on developing a schema 
import girder_client
import os,sys, json
import dagSecrets as dgs
from collections import Counter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# import pandas as pd
import pickle
import brain_region_maps as brm

%load_ext autoreload
%autoreload 2
import pandas as pd
import adrcSchemaHelpers as hlprs
import brain_region_maps as brn
gc = girder_client.GirderClient(apiUrl=dgs.cbApiUrl)
gc.authenticate(apiKey=dgs.cbApiKey)

from pivottablejs import pivot_ui

In [2]:
npPickleFile = 'npDemoData.pickle'
savePickle = True
reload = True

if os.path.isfile(npPickleFile) and not reload:
    with open(npPickleFile,"rb") as fp:
        npSlideDict=  pickle.load(fp)
else:
    print("RELOADING DATA")
    npSlideDict = hlprs.getADRCcollection(gc,rootCollectionName='ARMIN_Working_Cases')
    if savePickle:
        with open(npPickleFile,"wb") as fp:
            pickle.dump(npSlideDict,fp)

RELOADING DATA


In [20]:
dfd = []
needsMap = []

for k in npSlideDict:
    yr = int(k.split("_")[0])
    caseId = k.split("_")[1]
    caseSet = npSlideDict[k]
    if caseId not in brn.MAP:
        dfd.append({'Year':yr,'caseID':caseId,'totalSlidesInDir':len(caseSet),'validBrainMap':'No'})
        needsMap.append(caseId)
    elif caseId in brn.MAP:
        dfd.append({'Year':yr,'caseID':caseId,'totalSlidesInDir':len(caseSet),'validBrainMap':'Yes'})

In [21]:
#pivot_ui(pd.DataFrame(dfd))
print(len(needsMap))
needsMap

14


['E14-09',
 'OS-140327',
 'OS-140909',
 'OS-141217',
 'OSWH-140318',
 'E16-128',
 'E16-15',
 'E16-24',
 'E16-27',
 'E16-33',
 'E16-34',
 'OS',
 'OS',
 'OS-180213']

In [ ]:
regionList = []
for k in brm.MAP:
    regionList += brm.MAP[k].values()

In [ ]:
x= []

for k,v in Counter(regionList).most_common():
    if k not in brn.cleanupKeys and k not in brn.nonSchemaRegions:
    
        print(k,v)
        x.append(k)

In [ ]:
x

In [ ]:
## meta.region  meta.stain  meta.subject
basicSlideInfo = []
mdKeys = ['region','stain','subject','brain_region']

for subjyr in npSlideDict:
    for s in npSlideDict[subjyr]:
        slideData = {'folderId': subjyr,'slideName':s['name'],'itemId':s['_id']}
        
        for k in mdKeys:
            if k in s['meta']:
                slideData[k] = s['meta'][k]
        basicSlideInfo.append(slideData)
pd.DataFrame(basicSlideInfo).to_csv("adrcInitialMetadataExport.csv")

In [ ]:
npByYearCase = {}
for k in npSlideDict:
    try:
        (year,caseId) = k.split("_")
        if year not in npByYearCase:
            npByYearCase[year]= []
            
        npByYearCase[year].append(caseId)
        
    except:
        print("Weird pattern for",k)

In [ ]:
from ipywidgets import Dropdown, interactive, HBox, interactive_output
from collections import Counter
npYearList = list(npByYearCase.keys())

dpYear = Dropdown(options=npYearList,description='Year')
dpCaseId = Dropdown(options=npByYearCase[npYearList[0]],description='CaseId')


def on_change(*args):
    value = dpYear.value
    dpCaseId.options = npByYearCase[value]
dpYear.observe(on_change, names='value')

def analyzeNpCase(year, caseId):
    print("Gutman needs a bath")
    ### This will perform an analysis of a folder of images to see the number of slides 
    ## And check the metadata
    npKey = "%s_%s" % ( year,caseId)
    slideList = npSlideDict[npKey]
    
    ## Keys of interest
    koi = ['region','regionId','subject','stain','Stain','Region']
    slideKeys = {}
    for k in koi:
        slideKeys[k]= []
    
    caseMeta = [] ## 
    d=caseMeta ## Exposing this as a global
    
    for s in slideList:
        meta = s['meta']
        for k in meta:
            if k in koi:
                slideKeys[k].append(meta[k])
        caseMeta.append( hlprs.validateSlide(s,caseId) )
                
                
    ## Now go through and make things into counters so it's easier to see
    for k in koi:
        slideKeys[k] = Counter(slideKeys[k])
                
    print("Unique RegionId Count: %d \t Distinct SubjID: \t%d"% (len(slideKeys['region']),len(slideKeys['subject'])))
    print("Total slides found: %d" % len(slideList))  ## Will add in schema maps as well
    df = pd.DataFrame(caseMeta)
#     print(df)
    if 'region' in df.columns:
        print(df.region.value_counts())
    else:
        print("region tag not found..")
        print([s['name'] for s in slideList])

    
ui = HBox([dpYear,dpCaseId])
w = interactive_output(analyzeNpCase, {'year':dpYear, 'caseId':dpCaseId})
display(ui,w)

## Key I am interested in is npSchema going forward
## Need to figure out what to do with control slides..  will add a key to the Schema that's optional called controlSlide


In [ ]:
brm.MAP['A18-07']

In [ ]:
## LEt's examine a specific case...  2020_E20-106
import re

caseId = '2020_E20-106'
npSlideDict[caseId]

adrcName_re = re.compile('(?P<subject>.\d\d-\d+)_(?P<blockID>\d+) (?P<stain>.*)\.[svs|ndpi]')

## Let me look at the filenames in this dictionary
for slideName in [x['name'] for x in npSlideDict[caseId]]:
    m =  adrcName_re.search(slideName)
    if m:
        print(slideName,m.groupdict())
    elif ('CON' in slideName.upper() ):  ### This is probably a control slide
        print("Control slide?",slideName)
    else:
        print("Shit nothing works",slideName)

In [ ]:
brm.MAP['A20-11']